# Hypothesis Testing with *z*-scores and *t*-tests


## *z*-test

conducting a *t*-test with two independent samples

In [ ]:
# @title Click to hide code
from stats_project import RandomData
data = RandomData(groups = 1, n = 12)
data.z_test()

## One-Sample *t*-test

conducting a *t*-test with two independent samples

In [ ]:
# @title Click to hide code
from stats_project import RandomData
data = RandomData(groups = 1, n = 10)
data.one_sample_t_test()

## Independent-Samples *t*-test

conducting a *t*-test with two independent samples

In [ ]:
# @title Click to hide code
from stats_project import RandomData
data = RandomData(groups = 2, n = 10)
data.independent_samples_t_test()

## Dependent-Samples *t*-test

conducting a *t*-test with two independent samples

In [ ]:
# @title Click to hide code
from stats_project import RandomData
data = RandomData(groups = 2, n = 10)
data.dependent_samples_t_test()